<a href="https://colab.research.google.com/github/Akhil-ender/Fine-Tuning-LLM-s-for-Classification/blob/main/Distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys
# Mount the Google Drive to access files stored there
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/NLP/Class_notebooks/Custom_Funtions')

# Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import CustomPreprocessorSpacy as cp
import torch
import torch.nn as nn


# New libraries introduced in this notebook
import evaluate
from datasets import Dataset
from torch.utils.data import  DataLoader, RandomSampler, SequentialSampler
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers import pipeline
from datasets import load_dataset, load_metric
import numpy as np

import wandb

In [ ]:
model_folder = '/content/drive/MyDrive/NLP/Assignments/HW6/Distilbert'

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP/Assignments/HW5/train.csv')
test = pd.read_csv('/content/drive/MyDrive/NLP/Assignments/HW5/test.csv')

In [ ]:
X_train = train['Tweet']
X_test = test['Tweet']
y_train = train.drop(['ID','Tweet'],axis=1)
y_test = test.drop(['ID','Tweet'],axis=1)

In [ ]:
X_train

0       “Worry is a down payment on a problem you may ...
1       Whatever you decide to do make sure it makes y...
2       @Max_Kellerman  it also helps that the majorit...
3       Accept the challenges so that you can literall...
4       My roommate: it's okay that we can't spell bec...
                              ...                        
7719    @BadHombreNPS @SecretaryPerry If this didn't m...
7720    Excited to watch #stateoforigin tonight! Come ...
7721    Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi...
7722    #ThingsIveLearned The wise #shepherd never tru...
7723    I am really flattered and happy to hear those ...
Name: Tweet, Length: 7724, dtype: object

In [ ]:
# Making Sure all the labels are an int datatype
y_train = y_train.apply(lambda x: list(map(float, x)))

In [ ]:
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(float, x)))

In [ ]:
X_train_cleaned = cp.SpacyPreprocessor(model='en_core_web_sm', batch_size = 128).transform(X_train.values)
X_test_cleaned = cp.SpacyPreprocessor(model='en_core_web_sm', batch_size = 128).transform(X_test.values)

/content/drive/MyDrive/NLP/Class_notebooks/Custom_Funtions/CustomPreprocessorSpacy.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/content/drive/MyDrive/NLP/Class_notebooks/Custom_Funtions/CustomPreprocessorSpacy.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
DS = Dataset.from_dict({
    'texts': X_train_cleaned,
    'labels': y_train.values
})

testset = Dataset.from_dict({
    'texts': X_test_cleaned,
    'labels': y_test.values
})

In [ ]:
# Splitting the given train set into Trainset and Evaluation Set
train_eval_split = DS.train_test_split(test_size=0.2)
trainset = train_eval_split['train']
valset = train_eval_split['test']

In [ ]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["texts"],padding="max_length", truncation=True)

tokenized_trainsets = trainset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

In [ ]:
tokenized_valsets = valset.map(tokenize_function, batched=True)
tokenized_testsets = testset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
tokenized_trainsets.set_format(type="torch")
tokenized_valsets.set_format(type="torch")
tokenized_testsets.set_format(type="torch")

In [ ]:
from transformers import AutoModel
from transformers import DistilBertForSequenceClassification
#from transformers import

In [ ]:
# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=11)

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig

# Configuring the model for multi-label classification
config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=11)
model = DistilBertForSequenceClassification(config)


In [ ]:
config.id2label = {0:'anger', 1: 'anticipation',2: 'disgust',3: 'fear',4: 'joy',5: 'love',6: 'optimism',
  7:'pessimism', 8: 'sadness', 9: 'surprise', 10: 'trust'}

In [ ]:
config.label2id = {v: k for k, v in config.id2label .items()}

In [ ]:
config.problem_type = "multi_label_classification"

In [ ]:
config

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "initializer_range": 0.02,
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.39.3",
  "vocab_size": 30522
}

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.sigmoid(torch.tensor(logits)) > 0.5
    # Flatten to 1D arrays to compute multi-label accuracy or F1
    predictions_flat = predictions.flatten()
    labels_flat = labels.flatten()
    return {
        "f1": f1_score(labels_flat, predictions_flat, average="binary"),
        "accuracy": accuracy_score(labels_flat, predictions_flat),
    }


In [ ]:
# Configure training parameters
training_args = TrainingArguments(

    # Training-specific configurations
    num_train_epochs=10,
    per_device_train_batch_size=8, # Number of samples per training batch
    per_device_eval_batch_size=8, # Number of samples per validation batch
    weight_decay=0.1, # weight decay (L2 regularization)
    learning_rate=3e-5, # learning arte
    optim='adamw_torch', # optimizer
    remove_unused_columns=False, # flag to retain unused columns

    # Checkpoint saving and model evaluation settings
    output_dir=model_folder,  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 50 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=50,  # Save a model checkpoint every 50 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=1,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="accuracy",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations
    logging_strategy='steps',
    logging_steps=50,
    #report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='Distilbert-test',  # Experiment name for Weights & Biases
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_trainsets,
    eval_dataset=tokenized_valsets,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# specify the project name where the experiment will be logged
%env WANDB_PROJECT = nlp_course_spring_2024-HW6-distilbert

env: WANDB_PROJECT=nlp_course_spring_2024-HW6-distilbert


In [ ]:
trainer.train()

wandb: Currently logged in as: akhilender01 (akhilender). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1,Accuracy
50,0.499400,0.474549,0.001635,0.784466
100,0.474100,0.471499,0.000000,0.784407
150,0.482100,0.471824,0.000000,0.784407
200,0.476000,0.470320,0.030999,0.786643
250,0.480300,0.474306,0.150582,0.780936
300,0.472400,0.470031,0.197885,0.785819
350,0.482300,0.446781,0.212816,0.801942
400,0.443900,0.452851,0.391190,0.795057
450,0.440200,0.439291,0.303301,0.807532
500,0.439800,0.429889,0.413793,0.815946


TrainOutput(global_step=1250, training_loss=0.4179075912475586, metrics={'train_runtime': 1776.9746, 'train_samples_per_second': 34.773, 'train_steps_per_second': 4.35, 'total_flos': 1324224156380160.0, 'train_loss': 0.4179075912475586, 'epoch': 1.62})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3783750534057617,
 'eval_f1': 0.5603203737693977,
 'eval_accuracy': 0.8449543983524566,
 'eval_runtime': 43.0916,
 'eval_samples_per_second': 35.854,
 'eval_steps_per_second': 4.502,
 'epoch': 1.62}

In [ ]:
eval_results = trainer.evaluate(tokenized_valsets)

In [ ]:
#Log results to Wandb
wandb.log({"eval_accuracy": eval_results["eval_accuracy"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1"]})

In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 1100.


In [ ]:
model_folder

'/content/drive/MyDrive/NLP/Assignments/HW6/Distilbert'

In [ ]:
# Loading the saved checkpoint model
model_checkpoint = model_folder+'/checkpoint-'+best_model_checkpoint_step

In [ ]:
model_checkpoint

'/content/drive/MyDrive/NLP/Assignments/HW6/Distilbert/checkpoint-1100'

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_checkpoint)
model.to('cuda')
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_testsets,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
test_results = trainer.evaluate()

In [ ]:
wandb.log({"test_accuracy": test_results["eval_accuracy"], "test_loss": test_results["eval_loss"], "test_f1": test_results["eval_f1"]})

In [ ]:
wandb.finish()

eval/accuracy,▁▁▁▂▁▂▃▃▄▅▅▅▅▅▆▆▆▆▆▇▇█▇▇███
eval/f1,▁▁▁▁▃▃▄▆▅▆▆▇▆▅▇▇▇█▇▇▇██████
eval/loss,██████▆▆▅▅▄▄▄▄▃▃▂▃▄▂▃▁▂▁▁▁▁
eval/runtime,█▂▁▁▃▁▁▃▂▁▂▂▃▃▂▃▁▂▃▃▂▃▂▂▂▄▃
eval/samples_per_second,▁▇██▆██▆▆█▇▆▅▅▇▆█▇▅▆▆▆▆▆▆▄▅
eval/steps_per_second,▁▇██▆█▇▆▆█▇▆▅▅▇▆█▇▅▆▆▆▆▆▆▄▅
eval_accuracy,▁
eval_f1,▁
eval_loss,▁
test_accuracy,▁▁
test_f1,▁▁


## Model Inference

In [ ]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, config=config)

In [ ]:
pipe("I am sad that, RCB has never won an IPL")

[{'label': 'sadness', 'score': 0.7698304057121277}]

In [ ]:
test['Tweet'][1]

'Academy of Sciences, eschews the normally sober tone of scientific papers and calls the massive loss of wildlife a “biological annihilation'

In [ ]:
pipe(test['Tweet'][1])

[{'label': 'disgust', 'score': 0.4360220432281494}]

In [ ]:
pipe("Food was awesome at that restaraunt")

[{'label': 'joy', 'score': 0.6293020844459534}]

In [ ]:
pipe("Very sad that Dallas Cowboys have lost the game. Pain bro!")

[{'label': 'sadness', 'score': 0.8059729337692261}]